# **Load Dataframe**

In [ ]:
# # !pip install ipdb
# import pdb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/datalatih/Dataset.csv',delimiter = ';')

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# df = df.sort_values(by=[df.columns[4]]).reset_index(drop = 'true')

In [ ]:
import seaborn as sns

In [ ]:
cor1 = df.corr()
cor1

In [ ]:
df.loc[df[df.columns[4]]<10]

In [ ]:
flow_cut = 10
condition = [
             df[df.columns[4]] < flow_cut,
             df[df.columns[4]] >= flow_cut
]

choice = ['OFF','ON']

df['Status'] = np.select(condition,choice)

In [ ]:
# df.loc[df[df.columns[-1]]
X11 = df.copy()
X11.loc[(X11.Status == 'OFF'),'Status']= np.nan

In [ ]:
X11.isna().sum()

In [ ]:
data1 = df.copy()
fig, axes = plt.subplots(len(data1.columns), 1, figsize=(20,36))
for i,col in enumerate(data1):
  sns.scatterplot(x=data1.index,y=data1[data1.columns[i]],hue=data1.Status,edgecolor=None, ax=axes[i])

In [ ]:
data = X11.copy()
fig, axes = plt.subplots(len(data.columns), 1, figsize=(20,36))
for i,col in enumerate(data):
  sns.scatterplot(x=X11.index,y=X11[X11.columns[i]],hue=X11.Status,edgecolor=None, ax=axes[i])

In [ ]:
data = df.copy()
fig, axes = plt.subplots(len(data.columns), 1, figsize=(20,36))
for i,col in enumerate(data):
  sns.scatterplot(x=df.index[:200],y=df[df.columns[i]][:200],hue=df.Status[:200],edgecolor=None, ax=axes[i])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
MM = MinMaxScaler()

In [ ]:
!pip install fastdtw

In [ ]:
Dataset_ = df[df.columns[-3:-1]][:200]
Dataset_ = MM.fit_transform(Dataset_)
X10 = Dataset_[:,0]
Y10 = Dataset_[:,1]

In [ ]:
Y10

In [ ]:
X10.shape

In [ ]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw


In [ ]:
distance, path = fastdtw(X10, Y10, dist=euclidean)
print(distance)
print(path)

In [ ]:
# data = df.copy()
# fig, axes = plt.subplots(len(data.columns), 1, figsize=(20,36))
# # row_cut = 1700
# n_window = 20
# # dataroll = data.rolling(n_window).mean()
# # print(dataroll)
# for i,col in enumerate(data):
#     # print(data.rolling(n_window).mean()[col])
#     # axes[i].plot(data.rolling(n_window).mean()[col],c='red')
#     if data[data.columns[0]] <= 0.003:
#       axes[i].scatter(x = data.index, y = data[col],c='red')
#       axes[i].set_title(data.columns[i])
#     else:
#       axes[i].scatter(x = data.index, y = data[col],c='blue')
#       axes[i].set_title(data.columns[i])
#     # axes[i].plot(dataroll.iloc[row_cut:,[i]],c='blue')
#     # axes[i].set_title(data.columns[i])

In [ ]:
# X0 = data.rolling(n_window).mean()

In [ ]:
# # X1 = X0.dropna()
# X1 = data.copy()

In [ ]:
# X1

In [ ]:
# flow_cut = 0.0030
# X1.loc[X1[X1.columns[0]]<flow_cut]

In [ ]:
# condition = [
#              X1[X1.columns[0]] < flow_cut,
#              X1[X1.columns[0]] >= flow_cut
# ]

# choice = ['OFF','ON']

# X1['Status'] = np.select(condition,choice)

In [ ]:
# X1[X1.columns[4]].value_counts()

In [ ]:
cov = df.cov(min_periods=None)
cov

In [ ]:
X = df.iloc[:,[5,7,11,12]].reset_index(drop = 'True')
Y = df.iloc[:,[13]]

In [ ]:
X

In [ ]:
Y

# **Klasifikasi dengan Support Vector Machine**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0, stratify = Y)

In [ ]:
# Aplying Feature Scaling
from sklearn.preprocessing import MinMaxScaler
MM = MinMaxScaler()
X_train = pd.DataFrame(MM.fit_transform(X_train), columns=X.columns)
X_test = pd.DataFrame(MM.transform(X_test), columns=X.columns)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
pca.singular_values_

In [ ]:
fig = plt.figure(figsize=(15, 15), dpi=200)
ax = fig.add_subplot(projection='3d')
colors = {'OFF':'red', 'ON':'blue'}
ax.scatter(X_train[:,0], X_train[:,1], X_train[:,2],c = Y_train['Status'].apply(lambda x: colors[x]),marker = "^")
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('PCA 3')
plt.show()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X = [['OFF', 1], ['ON', 2]]

In [ ]:
# Y_train.value_counts()

In [ ]:
# Y_train = np.array(Y_train)
# Y_test = np.array(Y_test)

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'kernel': ('linear', 'rbf'), 'C':[0.1, 1, 10]}
svc = svm.SVC()

In [ ]:
np.squeeze(Y_train.values).shape

In [ ]:
Y_train.values

In [ ]:
# %pdb
clf = GridSearchCV(svc, parameters)
# clf = SVC(kernel = 'linear' , random_state = 0)
# clf = RFC()
# clf.fit(X_train, np.squeeze(Y_train.values))
clf.fit(X_train, Y_train.values.ravel())

In [ ]:
Y_train.values.shape

In [ ]:
sorted(clf.cv_results_.keys())

In [ ]:
# Predicting the Test set results
Y_pred = clf.predict(X_test)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
print(round(accuracy_score(Y_test,Y_pred)*100,2),'%')

# **Klustering dengan K-Means**

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
df

In [ ]:
df2 = df.copy()
X1 = df2.iloc[:,[11,12]]

In [ ]:
tmp = pd.concat([X1.copy(),Y],axis=1)
tmp.head()
tmp_OFF = tmp[tmp.Status=='OFF']
tmp_ON = tmp[tmp.Status=='ON']
tmp_OFF.describe()


In [ ]:
tmp_ON.describe()

In [ ]:
df

In [ ]:
threshold_cut = 700
df2 = df.loc[df[df.columns[12]] > threshold_cut].reset_index(drop='true')
X0 = np.array(df2.iloc[:,[11,12]])

In [ ]:
X0

In [ ]:
X = np.array([[320.11892761, 957.07405719],
              [386.82226138, 1166.21666667]], np.float64)

kmeans = KMeans(n_clusters=2,init = X, random_state=0, n_init = 1).fit(X0)
# kmeans2 =  = KMeans(n_clusters=2, random_state=0, n_init = 1).fit(X0)
print(kmeans.cluster_centers_)
print(kmeans.labels_)
print(kmeans.inertia_)
print(kmeans.n_iter_)

In [ ]:
Y0 = pd.DataFrame(kmeans.labels_, columns = ['Label'])

In [ ]:
X0

In [ ]:
Centroid_X = kmeans.cluster_centers_[:,[0]]
Centroid_Y = kmeans.cluster_centers_[:,[1]]

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=200)
plt.scatter(x = X0[:,[0]], y = X0[:,[1]], c = Y0.Label, marker = ".")
plt.scatter(Centroid_X, Centroid_Y, c='red', marker = "s")
plt.xlabel('Gross Load')
plt.ylabel('Main Steam Flow')
plt.show()

In [ ]:
# kmeans.predict()

In [ ]:
# import pickle
# pickle.dump(kmeans,open('kmeans.pkl','wb'))

In [ ]:
# !pip freeze > requirements.txt 

# **SINGULAR VALUE DECOMPOSITION**

In [ ]:
import scipy.linalg as la
import cmath
import cv2 as cv2

In [ ]:
df.iloc[:,:-1]

In [ ]:
A = MM.fit_transform(df.iloc[:,:-1])

In [ ]:
m,n = A.shape

In [ ]:
m

In [ ]:
def Sigmanew(m,n,Sig_):
  S = np.zeros((m,n))
  n_ = np.minimum(m,n)
  S[:n_,:n_] = np.diag(Sig_)
  return S

def Sigmanew_reduction(r,Sig_):
  S = np.zeros((r,r))
  S[:r,:r] = np.diag(Sig_[:r])
  return S

In [ ]:
U,Sig_,VT = np.linalg.svd(A)

In [ ]:
S = Sigmanew(m,n,Sig_)

In [ ]:
U@S@VT

In [ ]:
r = 4
S_red = Sigmanew_reduction(r,Sig_)
U_red = U[:,:r]
VT_red = VT[:r,:]

In [ ]:
U_red@S_red@VT_red